In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Add project 1 to sys path 
import sys
sys.path.append('../project1/')
import tools as proj1_tools

import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import Lasso

from IPython.display import clear_output

In [ ]:
import scipy.sparse as sp
np.random.seed(12)
import warnings
#Comment this to turn on warnings
warnings.filterwarnings('ignore')
### define Ising model params
# system size
L = 40
n = 10000
# create 10000 random Ising states
states=np.random.choice([-1, 1], size=(n,L))
from project2_tools import ising_energies1D 

# calculate Ising energies
energies=ising_energies1D(states)

In [ ]:
energies

In [ ]:
L = 40
n = 10000

s=np.random.choice([-1, 1], size=(n,L))

E = ising_energies1D(s)
X = np.einsum('...i,...j->...ij',s,s) 

In [ ]:
X_vectors = X.reshape(n,-1)

## One dimensional Ising model with linear regression

Fit the linear models and get the parameters; the coupling constant $J$

In [ ]:
# OLS
regr_ols = proj1_tools.Regression(X_vectors, E, solve_method = 'svd')
J_fit_ols = regr_ols.beta

# Ridge and Lasso
#lambdas = np.logspace(-10,0,11)
lambdas = np.array([1e-05,1e-04,1e-03,1e-02,1e-01])

J_fit_ridge = []
J_fit_lasso = []

for i,lmbd in enumerate(lambdas):
    print('Lambdas: ',i+1,'/',lambdas.shape[0])
    regr_ridge = proj1_tools.Regression(X_vectors, E, lmbd = lmbd, solve_method = 'svd')
    J_fit_ridge.append(regr_ridge.beta)
    regr_lasso = Lasso(alpha=lmbd, fit_intercept=False)
    J_fit_lasso.append(regr_lasso.fit(X_vectors, E).coef_)
    clear_output(wait=0.1)

### Plot

Parameters as matrix:

In [ ]:
vmin = np.min(J_fit_ols)
assert vmin < 0
# OLS
plt.figure(figsize=[3,3])
plt.title('OLS')
plt.imshow(J_fit_ols.reshape((L,L)), vmin = vmin, vmax = - vmin, cmap = plt.cm.bwr)
plt.show()
# Ridge and Lasso
for i,lmbd in enumerate(lambdas):
    print('Lambda = ',lmbd)
    fig,[ax1,ax2] = plt.subplots(1,2)
    ax1.imshow(J_fit_ridge[i].reshape((L,L)), vmin = vmin, vmax = - vmin, cmap = plt.cm.bwr)
    ax1.set_title('Ridge')
    ax2.imshow(J_fit_lasso[i].reshape((L,L)), vmin = vmin, vmax = - vmin, cmap = plt.cm.bwr)
    ax2.set_title('Lasso')
    plt.show()
# plt.colorbar(m)

Histogram of paramters:

In [ ]:
for i,lmbd in enumerate(lambdas):
    print('Lambda = ',lmbd)
    _,bins,_ = plt.hist(J_fit_ridge[i], bins = 30, histtype='step',density = True, label='ridge')
    _ = plt.hist(J_fit_ols, bins = bins, histtype='step', density = True, label='ols')
    _ = plt.hist(J_fit_lasso[i], bins = bins, histtype='step', density = True, label='lasso')
    plt.legend()
    plt.show()

In [ ]:
print(ridge_r2)
print(lasso_r2)

# TODO:

- Bias variance decomposition
- R2 and squared error (discussion)

In [ ]:
np.diag(J_fit_lasso[7].reshape((L,L)))

# TEST performance with mse and r2

In [ ]:
ridge_mse = []
ridge_r2 = []

lasso_mse = []
lasso_r2 = []

lambdas = np.array([1e-05,1e-04,1e-03,1e-02,1e-01])

for i,lmbd in enumerate(lambdas):
    print('Lambdas: ',i+1,'/',lambdas.shape[0])
    ridge = proj1_tools.Regression(X_vectors, E, lmbd = lmbd, solve_method = 'svd')
    r_pred = ridge.predict(test)
    lasso = Lasso(alpha=lmbd, fit_intercept=False)
    l_pred = lasso.predict(test)
    
    clear_output(wait=0.1)

In [ ]:
a=np.random.randint(0,9,5)
print(a,'\n')
b=np.random.randint(0,9,5)
print(b)

In [ ]:
np.einsum('...i,...j->...ij',a,b)

In [ ]:
energies.shape

In [ ]:
np.einsum('...i,...j->...ij')